#Project 4: Enhance the Travel Planning Agentic system using CrewAI by adding “real-time flight search” and “hotel search” agents

## Install all packages and import modules

In [ ]:


!pip install crewai langchain langchain-openai langchain-community python-dotenv google-search-results

import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from serpapi import GoogleSearch
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from datetime import datetime, timedelta
import json

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Enable API and Initialize LLM

In [ ]:
from google.colab import userdata

# Get API keys
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")
SERPAPI_API_KEY = userdata.get("SERPAPI_API_KEY")

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,
    openai_api_key=OPENAI_API_KEY
)

##Tools

### Tavily web search tool

In [ ]:
# Web search tool
search = TavilySearchResults(tavily_api_key=TAVILY_API_KEY, max_results=5)

@tool("Web Search")
def search_web(query: str) -> str:
    """Search the web for up-to-date destination info and return formatted results."""
    try:
        results = search.invoke(query)
        return "\n\n".join(
            f"Source: {r.get('url')}\nTitle: {r.get('title')}\nContent: {r.get('content')}"
            for r in results
        )
    except Exception as e:
        return f"Error in web search: {str(e)}"

/tmp/ipython-input-4271101756.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(tavily_api_key=TAVILY_API_KEY, max_results=5)


###Fina Airport Code Tool

**The Challenge:**

SerpAPI's Google Flights engine requires precise IATA airport codes (e.g., "BKK" not "Bangkok"), but users typically provide destinations as countries or city names (e.g., "Thailand" or "Bangkok").

**Solution:**

1. Itinerary-First Approach
Instead of immediately searching flights, the system first creates a complete preliminary itinerary that:

Identifies the optimal arrival gateway city (e.g., Bangkok for Thailand)
Determines the best departure gateway city (e.g., Chiang Mai for northern route)
Plans the logical city-to-city routing between them

2. IATA Code Resolution
The Flight Search Agent then uses the find_airport_code tool to convert city names into IATA codes

In [ ]:
@tool("Find Airport Code")
def find_airport_code(city_or_location: str) -> str:
    """Find IATA airport code for a specific city or location using SerpAPI."""
    try:
        # Expanded lookup for common cities
        common_airports = {
            "san francisco": "SFO", "los angeles": "LAX", "new york": "JFK",
            "chicago": "ORD", "miami": "MIA", "seattle": "SEA", "boston": "BOS",
            "hanoi": "HAN", "ho chi minh city": "SGN", "saigon": "SGN",
            "da nang": "DAD", "danang": "DAD", "hue": "HUI",
            "bangkok": "BKK", "chiang mai": "CNX", "phuket": "HKT",
            "tokyo": "NRT", "osaka": "KIX", "kyoto": "KIX",
            "seoul": "ICN", "busan": "PUS", "singapore": "SIN",
            "kuala lumpur": "KUL", "penang": "PEN", "langkawi": "LGK",
            "london": "LHR", "paris": "CDG", "amsterdam": "AMS", "dubai": "DXB",
            "hong kong": "HKG", "mumbai": "BOM", "delhi": "DEL", "goa": "GOI",
            "sydney": "SYD", "melbourne": "MEL", "cairns": "CNS",
            "toronto": "YYZ", "vancouver": "YVR", "montreal": "YUL",
            "rome": "FCO", "milan": "MXP", "venice": "VCE", "florence": "FLR",
            "madrid": "MAD", "barcelona": "BCN", "seville": "SVQ",
            "berlin": "BER", "munich": "MUC", "frankfurt": "FRA",
            "istanbul": "IST", "cairo": "CAI", "casablanca": "CMN",
            "marrakech": "RAK", "johannesburg": "JNB", "cape town": "CPT"
        }

        city_key = city_or_location.lower().strip()
        if city_key in common_airports:
            return f"{common_airports[city_key]} ({city_or_location})"

        # Use SerpAPI for airport code search
        search_params = {
            "engine": "google",
            "q": f"{city_or_location} airport code IATA",
            "api_key": os.environ["SERPAPI_API_KEY"]
        }

        search_result = GoogleSearch(search_params)
        results = search_result.get_dict()

        if "error" in results:
            return f"Error finding airport code for {city_or_location}: {results['error']}"

        # Extract airport codes from search results
        search_text = ""
        if "organic_results" in results:
            for result in results["organic_results"][:3]:
                search_text += result.get("snippet", "") + " "
                search_text += result.get("title", "") + " "

        if "answer_box" in results:
            search_text += results["answer_box"].get("answer", "") + " "

        if "knowledge_graph" in results:
            kg = results["knowledge_graph"]
            search_text += kg.get("description", "") + " "

        # Extract 3-letter airport codes
        import re
        airport_codes = re.findall(r'\b([A-Z]{3})\b', search_text)
        exclude_words = {'THE', 'AND', 'FOR', 'YOU', 'CAN', 'ALL', 'NEW', 'ONE', 'TWO', 'BUT', 'NOT', 'GET'}
        valid_codes = [code for code in airport_codes if code not in exclude_words]

        if valid_codes:
            return f"{valid_codes[0]} ({city_or_location})"
        else:
            return f"No airport code found for {city_or_location}"

    except Exception as e:
        return f"Error finding airport code for {city_or_location}: {str(e)}"

###Search Flight Tool

After having the IATA codes of all required airports, the Flight Search Agent will use this tool to search for flight at Serpapi using Serpapi API

In [ ]:
@tool("Search Flights")
def search_flights_detailed(origin_code: str, destination_code: str, departure_date: str, return_date: str = None, passengers: int = 1) -> str:
    """Search for detailed flight information with specific arrival and departure times."""
    try:
        flight_info = []
        flight_info.append(f"FLIGHT SEARCH RESULTS")
        flight_info.append(f"Route: {origin_code} -> {destination_code}")
        flight_info.append(f"Passengers: {passengers}")
        flight_info.append(f"Departure: {departure_date}" + (f" | Return: {return_date}" if return_date else " (One-way)"))
        flight_info.append("=" * 80)

        # Search outbound flights
        outbound_params = {
            "engine": "google_flights",
            "departure_id": origin_code,
            "arrival_id": destination_code,
            "outbound_date": departure_date,
            "currency": "USD",
            "hl": "en",
            "type": "2",
            "api_key": os.environ["SERPAPI_API_KEY"]
        }

        if return_date:
            outbound_params["return_date"] = return_date
            outbound_params["type"] = "1"

        search_result = GoogleSearch(outbound_params)
        results = search_result.get_dict()

        if "error" in results:
            return f"Flight search error: {results['error']}"

        # Process outbound flights with detailed timing
        flight_info.append("OUTBOUND FLIGHTS")
        flight_info.append("-" * 40)

        best_outbound_option = None
        if "best_flights" in results and results["best_flights"]:
            for i, flight in enumerate(results["best_flights"][:5], 1):
                price = flight.get("price", "N/A")
                duration = flight.get("total_duration", "N/A")
                flights = flight.get("flights", [])

                stops = len(flights) - 1
                stops_text = "Direct" if stops == 0 else f"{stops} stop{'s' if stops > 1 else ''}"

                if flights:
                    first_flight = flights[0]
                    last_flight = flights[-1]
                    airline = first_flight.get("airline", "N/A")
                    departure_time = first_flight.get("departure_airport", {}).get("time", "N/A")
                    arrival_time = last_flight.get("arrival_airport", {}).get("time", "N/A")
                    departure_airport = first_flight.get("departure_airport", {}).get("name", "N/A")
                    arrival_airport = last_flight.get("arrival_airport", {}).get("name", "N/A")

                    flight_info.append(f"Option {i}: ${price} | {duration} | {stops_text}")
                    flight_info.append(f"  {airline}")
                    flight_info.append(f"  Depart: {departure_time} from {departure_airport}")
                    flight_info.append(f"  Arrive: {arrival_time} at {arrival_airport}")

                    # Store best option for itinerary integration
                    if i == 1:  # Use first (best) option
                        best_outbound_option = {
                            "departure_time": departure_time,
                            "arrival_time": arrival_time,
                            "departure_airport": departure_airport,
                            "arrival_airport": arrival_airport,
                            "airline": airline,
                            "price": price
                        }

                flight_info.append("")

        # Search return flights with detailed timing
        best_return_option = None
        if return_date:
            flight_info.append("RETURN FLIGHTS")
            flight_info.append("-" * 40)

            return_params = {
                "engine": "google_flights",
                "departure_id": destination_code,
                "arrival_id": origin_code,
                "outbound_date": return_date,
                "currency": "USD",
                "hl": "en",
                "type": "2",
                "api_key": os.environ["SERPAPI_API_KEY"]
            }

            return_search = GoogleSearch(return_params)
            return_results = return_search.get_dict()

            if "best_flights" in return_results and return_results["best_flights"]:
                for i, flight in enumerate(return_results["best_flights"][:5], 1):
                    price = flight.get("price", "N/A")
                    duration = flight.get("total_duration", "N/A")
                    flights = flight.get("flights", [])

                    stops = len(flights) - 1
                    stops_text = "Direct" if stops == 0 else f"{stops} stop{'s' if stops > 1 else ''}"

                    if flights:
                        first_flight = flights[0]
                        last_flight = flights[-1]
                        airline = first_flight.get("airline", "N/A")
                        departure_time = first_flight.get("departure_airport", {}).get("time", "N/A")
                        arrival_time = last_flight.get("arrival_airport", {}).get("time", "N/A")
                        departure_airport = first_flight.get("departure_airport", {}).get("name", "N/A")
                        arrival_airport = last_flight.get("arrival_airport", {}).get("name", "N/A")

                        flight_info.append(f"Option {i}: ${price} | {duration} | {stops_text}")
                        flight_info.append(f"  {airline}")
                        flight_info.append(f"  Depart: {departure_time} from {departure_airport}")
                        flight_info.append(f"  Arrive: {arrival_time} at {arrival_airport}")

                        # Store best return option
                        if i == 1:
                            best_return_option = {
                                "departure_time": departure_time,
                                "arrival_time": arrival_time,
                                "departure_airport": departure_airport,
                                "arrival_airport": arrival_airport,
                                "airline": airline,
                                "price": price
                            }

                    flight_info.append("")

        # Add flight timing summary for itinerary integration
        flight_info.append("RECOMMENDED FLIGHT TIMES FOR ITINERARY INTEGRATION")
        flight_info.append("-" * 60)
        if best_outbound_option:
            flight_info.append(f"ARRIVAL: {best_outbound_option['arrival_time']} on {departure_date}")
            flight_info.append(f"Airport: {best_outbound_option['arrival_airport']}")
        if best_return_option:
            flight_info.append(f"DEPARTURE: {best_return_option['departure_time']} on {return_date}")
            flight_info.append(f"Airport: {best_return_option['departure_airport']}")

        return "\n".join(flight_info)

    except Exception as e:
        return f"Flight search error: {str(e)}"

###Search Hotel Tool

Search Hotel Tool is used by Accommodation Expert to look for hotels based on itinerary and budget

In [ ]:

@tool("Search Hotels")
def search_hotels(destination: str, check_in_date: str, check_out_date: str, guests: int = 2) -> str:
    """Search for hotel options using SerpAPI Google Hotels."""
    try:
        params = {
            "engine": "google_hotels",
            "q": destination,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": guests,
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": os.environ["SERPAPI_API_KEY"]
        }

        search_result = GoogleSearch(params)
        results = search_result.get_dict()

        if "properties" not in results:
            return f"No hotels found for {destination} on {check_in_date} to {check_out_date}"

        hotel_info = []
        hotel_info.append(f"HOTEL OPTIONS IN {destination.upper()}")
        hotel_info.append(f"Check-in: {check_in_date} | Check-out: {check_out_date} | Guests: {guests}")
        hotel_info.append("=" * 60)

        for i, hotel in enumerate(results.get("properties", [])[:8], 1):
            name = hotel.get("name", "Unknown Hotel")
            price = hotel.get("rate_per_night", {}).get("lowest", "N/A")
            rating = hotel.get("overall_rating", "N/A")
            reviews = hotel.get("reviews", "N/A")
            location = hotel.get("neighborhood", "City center")
            amenities = ", ".join(hotel.get("amenities", [])[:3])

            hotel_info.append(f"{i}. {name}")
            hotel_info.append(f"   Price: ${price}/night | Rating: {rating} ({reviews} reviews)")
            hotel_info.append(f"   Location: {location}")
            hotel_info.append(f"   Amenities: {amenities}")
            hotel_info.append("")

        return "\n".join(hotel_info)

    except Exception as e:
        return f"Hotel search error: {str(e)}"

##Define Agents

###PHASE 1: these agents are agents work together to create an initial plan in order to have destination cities and last places for the Flight Search Agent to look for IATA codes

In [ ]:

destination_researcher = Agent(
    role="Destination Research Expert",
    goal="Research comprehensive destination information and identify the main gateway cities for international travel.",
    backstory="Expert travel researcher with deep knowledge of global destinations, airport hubs, and optimal travel routes.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True,
)

attractions_specialist = Agent(
    role="Attractions & Activities Expert",
    goal="Discover and recommend the best attractions, activities, and experiences based on traveler preferences.",
    backstory="Local expert who specializes in finding both popular attractions and hidden gems tailored to traveler interests.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

city_route_planner = Agent(
    role="City Route Planning Specialist",
    goal="Plan the optimal route between cities and identify the best arrival and departure cities for international flights.",
    backstory="Expert in multi-city travel logistics who determines the most efficient routing and identifies gateway cities for flight booking.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

preliminary_itinerary_planner = Agent(
    role="Preliminary Itinerary Planner",
    goal="Create a detailed day-by-day itinerary with specific cities and timing, clearly identifying arrival and departure cities.",
    backstory="Professional travel planner who creates detailed itineraries and identifies exact cities for flight routing.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)


### PHASE 2: Flight Search Agent (runs AFTER itinerary identifies cities)

In [ ]:


flight_specialist = Agent(
    role="Flight Search Specialist",
    goal="Search for flights based on the cities identified in the itinerary, then provide exact arrival and departure times.",
    backstory="Aviation expert who finds optimal flights once the exact travel cities are determined from the completed itinerary.",
    tools=[find_airport_code, search_flights_detailed],
    llm=llm,
    memory=True,
    verbose=True
)

### PHASE 3: Final Integration Agent

In [ ]:

final_itinerary_integrator = Agent(
    role="Master Itinerary Integrator",
    goal="Take the preliminary itinerary and flight information to create a final time-integrated schedule with exact flight times.",
    backstory="Expert at combining flight schedules with travel itineraries to create perfectly timed, hour-by-hour travel plans.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

accommodation_specialist = Agent(
    role="Accommodation Expert",
    goal="Find the best hotels and accommodations that match traveler preferences and the finalized itinerary.",
    backstory="Hospitality expert who specializes in matching travelers with perfect accommodations in ideal locations.",
    tools=[search_hotels],
    llm=llm,
    memory=True,
    verbose=True
)

local_guide = Agent(
    role="Local Culture & Practical Guide",
    goal="Provide essential local knowledge including customs, etiquette, food, transportation, and practical tips.",
    backstory="Street-smart local expert with deep cultural knowledge and practical travel wisdom.",
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

travel_manager = Agent(
    role="Travel Project Manager",
    goal="Compile all information into a comprehensive, time-integrated travel guide.",
    backstory="Experienced travel project manager who creates the final polished travel plan with perfect timing integration.",
    llm=llm,
    memory=True,
    verbose=True
)

##Define Collaborative Tasks

Tasks are assigned to agents. Each agent completes its task using reasoning, memory, and tools. CrewAI coordinates these agents autonomously.

In [ ]:

def create_smart_adaptive_tasks(origin_city, destination, travel_dates, duration_days, preferences, budget, travelers_count):
    """Create tasks that first plan itinerary, then search flights, then integrate timing"""

    # Parse travel dates
    if " to " in travel_dates:
        departure_date, return_date = travel_dates.split(" to ")
        departure_date = departure_date.strip()
        return_date = return_date.strip()
    else:
        departure_date = travel_dates.strip()
        try:
            dep_date = datetime.strptime(departure_date, '%Y-%m-%d')
            return_date = (dep_date + timedelta(days=duration_days)).strftime('%Y-%m-%d')
        except:
            return_date = None

    return [
        # PHASE 1: ITINERARY PLANNING (determine cities first)

        Task(
            description=f"""
            Research {destination} comprehensively and identify key information:

            1. If {destination} is a country, identify the main gateway cities for international flights
            2. Research weather, culture, safety, and practical information
            3. Determine the best cities/regions to visit based on {preferences} and {duration_days} days
            4. Identify transportation options between cities

            **CRITICAL**: If the destination is a country (like "Thailand", "Vietnam", "Japan"), clearly identify:
            - Main international airport gateway cities (e.g., "Bangkok and Chiang Mai for Thailand")
            - Which city should be the arrival point for international flights
            - Which city should be the departure point for international flights

            This information is essential for flight booking later.
            """,
            agent=destination_researcher,
            expected_output="Comprehensive destination research with clearly identified gateway cities for flight routing."
        ),

        Task(
            description=f"""
            Based on the destination research, discover attractions and activities in {destination}:

            - Focus on {preferences} interests
            - Consider {budget} budget level
            - Account for {duration_days} days duration
            - {travelers_count} traveler(s)

            Include:
            - Must-see attractions in each recommended city
            - Hidden gems and local favorites
            - Activity timing and duration requirements
            - Transportation between attractions
            - Costs and booking requirements

            Organize recommendations by city/region to help with routing.
            """,
            agent=attractions_specialist,
            expected_output="City-organized attractions and activities list with timing and logistics information."
        ),

        Task(
            description=f"""
            Plan the optimal multi-city route for {destination}:

            Using the destination research and attractions, determine:
            1. **ARRIVAL CITY**: Best city for international flight arrival
            2. **DEPARTURE CITY**: Best city for international flight departure
            3. Optimal route between cities (if multi-city)
            4. Transportation methods and timing between cities
            5. How many days to spend in each city

            Consider:
            - Logical travel flow (minimize backtracking)
            - Transportation efficiency
            - Airport locations and flight availability
            - Travel time between cities

            **OUTPUT MUST CLEARLY STATE**:
            - "RECOMMENDED ARRIVAL CITY: [City Name]"
            - "RECOMMENDED DEPARTURE CITY: [City Name]"
            - City-by-city routing with days allocation
            """,
            agent=city_route_planner,
            expected_output="Complete multi-city routing plan with clearly identified arrival and departure cities for flight booking."
        ),

        Task(
            description=f"""
            Create a detailed preliminary itinerary for {destination}:

            Trip Details:
            - Duration: {duration_days} days
            - Start: {departure_date}
            - End: {return_date if return_date else 'calculated from duration'}
            - Travelers: {travelers_count}
            - Budget: {budget}
            - Preferences: {preferences}

            **CRITICAL REQUIREMENTS**:
            1. Use the city routing plan to structure the itinerary
            2. **Clearly state which city travelers arrive in on Day 1**
            3. **Clearly state which city travelers depart from on final day**
            4. Include specific timings for activities (morning/afternoon/evening)
            5. Include transportation between cities with timing
            6. Plan realistic daily schedules

            Format each day as:
            **Day X (Date) - [City Name]**
            - Morning (9:00 AM - 12:00 PM): [Activities]
            - Afternoon (12:00 PM - 6:00 PM): [Activities]
            - Evening (6:00 PM - 10:00 PM): [Activities]

            **MUST INCLUDE**:
            - Day 1: "Arrive in [CITY NAME]"
            - Final Day: "Depart from [CITY NAME]"

            This itinerary will be used to book flights to the correct cities.
            """,
            agent=preliminary_itinerary_planner,
            expected_output="Detailed day-by-day preliminary itinerary clearly showing arrival city (Day 1) and departure city (final day) for flight booking."
        ),

        # PHASE 2: FLIGHT SEARCH (now we know which cities!)

        Task(
            description=f"""
            Based on the completed preliminary itinerary, search for optimal flights:

            **FLIGHT ROUTING FROM ITINERARY**:
            1. Extract the ARRIVAL CITY from Day 1 of the itinerary
            2. Extract the DEPARTURE CITY from the final day of the itinerary
            3. Find IATA airport codes for:
               - Origin: {origin_city}
               - Destination arrival city: [from itinerary Day 1]
               - Departure city: [from itinerary final day]

            **SEARCH PARAMETERS**:
            - Outbound: {origin_city} → [Arrival city] on {departure_date}
            - Return: [Departure city] → {origin_city} on {return_date if return_date else 'end date from itinerary'}
            - Passengers: {travelers_count}
            - Budget: {budget}

            **PROCESS**:
            1. Use itinerary to identify exact cities
            2. Find IATA codes for all required airports
            3. Search comprehensive flight options
            4. **Provide exact arrival and departure times for integration**

            **OUTPUT REQUIREMENTS**:
            - Exact arrival time on Day 1: "Arrive at [time] at [airport]"
            - Exact departure time on final day: "Depart at [time] from [airport]"
            - Multiple flight options with detailed timing
            - Booking recommendations
            """,
            agent=flight_specialist,
            expected_output="Complete flight search with exact arrival/departure times based on itinerary cities, ready for final integration."
        ),

        # PHASE 3: FINAL INTEGRATION

        Task(
            description=f"""
            **CREATE THE FINAL TIME-INTEGRATED ITINERARY**

            Combine the preliminary itinerary with the exact flight times to create a perfect schedule:

            **INTEGRATION REQUIREMENTS**:
            1. Take Day 1 from preliminary itinerary and integrate with flight arrival time
            2. Take final day from preliminary itinerary and integrate with flight departure time
            3. Adjust all activities based on realistic timing from flights
            4. Account for airport transfers, customs, and jet lag

            **FINAL FORMAT MUST BE**:

            **DAY 1 ([Date]) - ARRIVAL**
            **FLIGHT ARRIVAL: [exact time] at [airport name]**
            **Morning ([time range]):**
            - [Airport transfer and activities]
            **Afternoon ([time range]):**
            - [Activities based on arrival time]
            **Evening ([time range]):**
            - [Activities and rest]

            **DAY 2-N ([Dates]) - [City Names]**
            [Continue with detailed morning/afternoon/evening breakdown]

            **DAY {duration_days} ([Date]) - DEPARTURE**
            **Morning ([time range]):**
            - [Final activities]
            **Afternoon ([time range]):**
            - [Airport transfer and departure prep]
            **FLIGHT DEPARTURE: [exact time] from [airport name]**

            **CRITICAL**: Every day must have specific times and realistic scheduling that accounts for the actual flight arrival and departure times.
            """,
            agent=final_itinerary_integrator,
            expected_output="Complete time-integrated itinerary from flight arrival to flight departure with hour-by-hour scheduling."
        ),

        # SUPPORTING TASKS

        Task(
            description=f"""
            Find accommodation for the finalized itinerary:
            - Dates: {departure_date} to {return_date if return_date else 'based on itinerary'}
            - Guests: {travelers_count}
            - Budget: {budget}
            - Cities: Based on the final integrated itinerary

            Provide specific hotel recommendations with:
            - Optimal locations based on planned activities
            - Pricing and booking information
            - Check-in/check-out logistics
            """,
            agent=accommodation_specialist,
            expected_output="Hotel recommendations perfectly aligned with the final itinerary cities and timing."
        ),

        Task(
            description=f"""
            Provide comprehensive local knowledge for {destination}:
            - Cultural customs and etiquette
            - Transportation systems and tips
            - Food recommendations and dining etiquette
            - Language phrases and communication tips
            - Safety information and emergency contacts
            - Local laws and cultural considerations
            - Money, tipping, and payment customs

            Focus on practical advice for {travelers_count} traveler(s) visiting multiple cities.
            """,
            agent=local_guide,
            expected_output="Complete practical guide for navigating the destination like a local."
        ),

        # FINAL COMPILATION

        Task(
            description="""
            Compile all information into the final comprehensive travel plan:

            **EXACT STRUCTURE REQUIRED**:

            # 🌍 COMPLETE TRAVEL PLAN

            ## ✈️ FLIGHT DETAILS
            [Complete flight information with exact times and booking details]

            ## 📍 DESTINATION OVERVIEW
            [Research summary and gateway cities explanation]

            ## 🎯 ATTRACTIONS & ACTIVITIES
            [Organized by city with timing and logistics]

            ## 🗓️ **DETAILED TIME-BASED ITINERARY**
            [Hour-by-hour schedule from flight arrival to flight departure]
            **Use this exact format for each day:**
            **DAY X (Date) - City**
            **Morning (X:XX AM - X:XX PM):** [specific activities]
            **Afternoon (X:XX PM - X:XX PM):** [specific activities]
            **Evening (X:XX PM - X:XX PM):** [specific activities]

            ## 🏨 ACCOMMODATION
            [Hotel recommendations with booking details]

            ## 🧭 LOCAL GUIDE
            [Cultural tips and practical information]

            ## 💡 FINAL TRAVEL TIPS
            [Booking recommendations, packing tips, and final advice]

            **This must be a complete, actionable travel guide that someone can follow hour by hour!**
            """,
            agent=travel_manager,
            expected_output="Complete professional travel plan with perfect flight-itinerary integration and detailed hour-by-hour scheduling."
        )
    ]

##Get User's inputs

In [ ]:
    origin_city = input("Your departure city: ") or "San Francisco"
    destination = input("Destination (can be country or city): ")
    travel_dates = input("Travel dates (YYYY-MM-DD or YYYY-MM-DD to YYYY-MM-DD): ") or "2025-03-15 to 2025-03-22"
    duration_days = int(input("Trip duration (days): ") or "7")
    travelers_count = int(input("Number of travelers: ") or "2")
    preferences = input("Travel preferences: ") or "Culture, Food, Temples, Beaches"
    budget = input("Budget level (budget/moderate/luxury): ") or "moderate"

Your departure city: Ho Chi Minh
Destination (can be country or city): China
Travel dates (YYYY-MM-DD or YYYY-MM-DD to YYYY-MM-DD): 2025-09-10 to 2025-09-20
Trip duration (days): 11
Number of travelers: 1
Travel preferences: adventure
Budget level (budget/moderate/luxury): moderate


## Create smart adaptive tasks

In [ ]:

tasks = create_smart_adaptive_tasks(
    origin_city, destination, travel_dates,
    duration_days, preferences, budget, travelers_count
)

## Create crew with proper task sequencing

In [ ]:

crew = Crew(
    agents=[
        # Phase 1: Itinerary planning (determine cities)
        destination_researcher,
        attractions_specialist,
        city_route_planner,
        preliminary_itinerary_planner,

        # Phase 2: Flight search (based on identified cities)
        flight_specialist,

        # Phase 3: Integration and support
        final_itinerary_integrator,
        accommodation_specialist,
        local_guide,
        travel_manager
    ],
    tasks=tasks,
    verbose=True,
    process=Process.sequential,  # Critical: ensures proper order
    memory=True
)


In [ ]:
results = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 624de877-b3e6-4c3f-a265-3bb7107915ea                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Destination Research Expert                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│              Research China comprehensively and identify key information:                                       │
│                                                                                                                 │
│              1. If China is a country, identify the main gateway cities for international flights               │
│              2. Research weather, culture, safety, and practical information                                    │
│              3. Determine the best cities/regions to visit based on adventure and 11 days                       │
│              4. Identify transportation options between cities                                                  │
│                                                                                                                 │
│              **CRITICAL**: If the destination is a country (like "Thailand", "Vietnam", "Japan"), clearly       │
│  identify:                                                                                                      │
│              - Main international airport gateway cities (e.g., "Bangkok and Chiang Mai for Thailand")          │
│              - Which city should be the arrival point for international flights                                 │
│              - Which city should be the departure point for international flights                               │
│                                                                                                                 │
│              This information is essential for flight booking later.                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Destination Research Expert                                                                             │
│                                                                                                                 │
│  Thought: Thought: I need to gather comprehensive information about China, including its main international     │
│  gateway cities, weather, culture, safety, practical information, top destinations to visit for adventure, and  │
│  transportation options between cities.                                                                         │
│                                                                                                                 │
│  Using Tool: Web Search                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Project Manager                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # 🌍 COMPLETE TRAVEL PLAN                                                                                      │
│                                                                                                                 │
│  ## ✈️ FLIGHT DETAILS                                                                                            │
│  - **Arrival Flight:**                                                                                          │
│    - **Option 1:** $194, 1 stop via Shenzhen                                                                    │
│      - Depart: 02:20 from Tan Son Nhat International Airport (SGN)                                              │
│      - Arrive: 13:25 at Beijing Capital International Airport (PEK)                                             │
│                                                                                                                 │
│  - **Return Flight:**                                                                                           │
│    - **Option 1:** $100, 1 stop via Shenzhen                                                                    │
│      - Depart: 14:30 from Beijing Capital International Airport (PEK)                                           │
│      - Arrive: 01:20 on 2025-09-21 at Tan Son Nhat International Airport (SGN)                                  │
│                                                                                                                 │
│  ## 📍 DESTINATION OVERVIEW                                                                                     │
│  China offers a diverse cultural and adventurous experience with rich heritage sites and stunning landscapes.   │
│  Major international gateway cities include Beijing and Shanghai, known for their extensive connectivity. This  │
│  travel plan covers Beijing, Xi’an, Zhangjiajie, Chengdu, Guilin/Yangshuo, and concludes in Shanghai.           │
│                                                                                                                 │
│  ## 🎯 ATTRACTIONS & ACTIVITIES                                                                                 │
│  - **Beijing:** Great Wall (Mutianyu), Forbidden City, Temple of Heaven, Summer Palace.                         │
│  - **Xi’an:** Terracotta Warriors, Xi'an City Wall, Muslim Quarter.                                             │
│  - **Zhangjiajie:** Zhangjiajie National Forest Park, Avatar Mountains, Glass Bridge, Tianmen Mountain.         │
│  - **Chengdu:** Chengdu Panda Breeding Research Base, Leshan Giant Buddha, Jinli Ancient Street.                │
│  - **Guilin/Yangshuo:** Li River Cruise, Moon Hill, Reed Flute Cave.                                            │
│                                                                                                                 │
│  ## 🗓️ **DETAILED TIME-BASED ITINERARY**                                                                         │
│                                                                                                                 │
│  **DAY 1 (2025-09-10) - Beijing**                                                                               │
│  **Afternoon (1:30 PM - 6:00 PM):** Arrive, clear customs, transfer to hotel, visit Tiananmen Square and        │
│  Forbidden City.                                        

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cf966c54-7ef3-4904-b8c8-6a00357eb6bd                                                                     │
│  Agent: Travel Project Manager                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 624de877-b3e6-4c3f-a265-3bb7107915ea                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # 🌍 COMPLETE TRAVEL PLAN                                                                        │
│                                                                                                                 │
│  ## ✈️ FLIGHT DETAILS                                                                                            │
│  - **Arrival Flight:**                                                                                          │
│    - **Option 1:** $194, 1 stop via Shenzhen                                                                    │
│      - Depart: 02:20 from Tan Son Nhat International Airport (SGN)                                              │
│      - Arrive: 13:25 at Beijing Capital International Airport (PEK)                                             │
│                                                                                                                 │
│  - **Return Flight:**                                                                                           │
│    - **Option 1:** $100, 1 stop via Shenzhen                                                                    │
│      - Depart: 14:30 from Beijing Capital International Airport (PEK)                                           │
│      - Arrive: 01:20 on 2025-09-21 at Tan Son Nhat International Airport (SGN)                                  │
│                                                                                                                 │
│  ## 📍 DESTINATION OVERVIEW                                                                                     │
│  China offers a diverse cultural and adventurous experience with rich heritage sites and stunning landscapes.   │
│  Major international gateway cities include Beijing and Shanghai, known for their extensive connectivity. This  │
│  travel plan covers Beijing, Xi’an, Zhangjiajie, Chengdu, Guilin/Yangshuo, and concludes in Shanghai.           │
│                                                                                                                 │
│  ## 🎯 ATTRACTIONS & ACTIVITIES                                                                                 │
│  - **Beijing:** Great Wall (Mutianyu), Forbidden City, Temple of Heaven, Summer Palace.                         │
│  - **Xi’an:** Terracotta Warriors, Xi'an City Wall, Muslim Quarter.                                             │
│  - **Zhangjiajie:** Zhangjiajie National Forest Park, Avatar Mountains, Glass Bridge, Tianmen Mountain.         │
│  - **Chengdu:** Chengdu Panda Breeding Research Base, Leshan Giant Buddha, Jinli Ancient Street.                │
│  - **Guilin/Yangshuo:** Li River Cruise, Moon Hill, Reed Flute Cave.                                            │
│                                                                                                                 │
│  ## 🗓️ **DETAILED TIME-BASED ITINERARY**                                                                         │
│                                                                                                                 │
│  **DAY 1 (2025-09-10) - Beijing**                                                                               │
│  **Afternoon (1:30 PM - 6:00 PM):** Arrive, clear custo

##View Final Output

In [ ]:

print("\n\n=== FINAL RESULT ===\n")
print(results)




=== FINAL RESULT ===

# 🌍 COMPLETE TRAVEL PLAN

## ✈️ FLIGHT DETAILS
- **Arrival Flight:**  
  - **Option 1:** $194, 1 stop via Shenzhen  
    - Depart: 02:20 from Tan Son Nhat International Airport (SGN)  
    - Arrive: 13:25 at Beijing Capital International Airport (PEK)  

- **Return Flight:**  
  - **Option 1:** $100, 1 stop via Shenzhen  
    - Depart: 14:30 from Beijing Capital International Airport (PEK)  
    - Arrive: 01:20 on 2025-09-21 at Tan Son Nhat International Airport (SGN)  

## 📍 DESTINATION OVERVIEW
China offers a diverse cultural and adventurous experience with rich heritage sites and stunning landscapes. Major international gateway cities include Beijing and Shanghai, known for their extensive connectivity. This travel plan covers Beijing, Xi’an, Zhangjiajie, Chengdu, Guilin/Yangshuo, and concludes in Shanghai.

## 🎯 ATTRACTIONS & ACTIVITIES
- **Beijing:** Great Wall (Mutianyu), Forbidden City, Temple of Heaven, Summer Palace.
- **Xi’an:** Terracotta Warriors, Xi